In [14]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_PYTHON'] = 'python'

spark = SparkSession.builder.appName("Word Count with Spark").getOrCreate()

def basic_word_count(book):
    word_count = (book.flatMap(lambda line: line.value.split(" ")).filter(lambda word: word != "").map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b))
    return word_count
    
book1 = spark.read.text(r'D:\ESDS\SEMESTER3\DIC\HW3\Books\AdvofSherlockHolmes.txt').rdd
book2 = spark.read.text(r'D:\ESDS\SEMESTER3\DIC\HW3\Books\AliceInWonderland.txt').rdd

word_count1 = basic_word_count(book1)
word_count2 = basic_word_count(book2)

word_count1.saveAsTextFile(r'D:\ESDS\SEMESTER3\DIC\HW3\Output\BasicWordCount\sherlock_wordcount')
word_count2.saveAsTextFile(r'D:\ESDS\SEMESTER3\DIC\HW3\Output\BasicWordCount\alice_wordcount')


In [25]:
def split(line):
    try:
        word, count = line.strip("[]()").split(", ")
        word = word.strip("'\"")
        count = int(count)
        return word, count
    except ValueError:
        return None


def top_25_words(book):
    words = (spark.sparkContext.textFile(book).map(split).sortBy(lambda x: x[1], ascending=False).take(25))
    return words

top_25_sherlock = top_25_words(r'D:\ESDS\SEMESTER3\DIC\HW3\Output\BasicWordCount\sherlock_wordcount\part-00000')
top_25_alice = top_25_words(r'D:\ESDS\SEMESTER3\DIC\HW3\Output\BasicWordCount\alice_wordcount\part-00000')

print("Top 25 Common Words in Sherlock Holmes:")
for word, count in top_25_sherlock:
    print(f"{word}: {count}")

print("\nTop 25 words in Alice in Wonderland:")
for word, count in top_25_alice:
    print(f"{word}: {count}")


Top 25 Common Words in Sherlock Holmes:
the: 5412
and: 2794
of: 2724
to: 2702
a: 2575
I: 2533
in: 1706
that: 1557
was: 1361
his: 1096
is: 1073
you: 1034
he: 1014
it: 977
my: 901
have: 894
with: 843
had: 807
as: 776
which: 753
at: 737
for: 698
be: 610
not: 605
from: 485

Top 25 words in Alice in Wonderland:
the: 1683
and: 782
to: 778
a: 667
of: 604
she: 485
said: 416
in: 406
it: 357
was: 329
you: 306
I: 249
as: 246
that: 225
Alice: 221
with: 214
at: 209
her: 204
had: 176
all: 168
be: 153
on: 148
for: 147
or: 136
not: 129
